In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
import json

In [2]:
features = pd.read_csv('../data/interim/dc_311_preprocessed_data.csv')

/var/folders/rq/rs89ll8n2w93x8gnl0mtn7sw0000gn/T/ipykernel_20767/260465707.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  features = pd.read_csv('../data/interim/dc_311_preprocessed_data.csv')


In [3]:
features.head()

,objectid,servicecode,servicecodedescription,servicetypecodedescription,organizationacronym,servicecallcount,adddate,resolutiondate,serviceduedate,serviceorderdate,...,gis_id,globalid,creator,created,editor,edited,gdb_from_date,gdb_to_date,gdb_archive_oid,days_to_resolve
0,9591692,S0031,Bulk Collection,SWMA- Solid Waste Management Admistration,DPW,1,2021-01-01 05:32:05,2021-01-06 15:34:42,2021-01-06 23:59:00,2021-01-01 05:32:05,...,NaN,{B7D1802E-75AA-7CDF-E053-532F520AE3B3},NaN,NaN,NaN,NaN,1.725867e+12,2.534023e+14,1873518.0,5.0
1,9591693,BICYCLE,Abandoned Bicycle,SWMA- Solid Waste Management Admistration,DPW,1,2021-01-01 05:06:02,2021-01-01 05:07:01,2021-01-09 04:59:00,2021-01-01 05:06:02,...,NaN,{B7D1802E-75AB-7CDF-E053-532F520AE3B3},NaN,NaN,NaN,NaN,1.725867e+12,2.534023e+14,1873519.0,0.0
2,9591695,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,DDOT,1,2021-01-01 05:33:08,2021-01-05 05:55:04,2021-01-12 05:33:08,2021-01-01 05:33:08,...,NaN,{B7D1802E-75AD-7CDF-E053-532F520AE3B3},NaN,NaN,NaN,NaN,1.725867e+12,2.534023e+14,1873521.0,4.0
3,9591696,S0031,Bulk Collection,SWMA- Solid Waste Management Admistration,DPW,1,2021-01-01 11:58:18,2021-01-04 12:54:36,2021-01-05 04:59:00,2021-01-01 11:58:18,...,NaN,{B7D764C0-4508-BB8F-E053-532F520A63E8},NaN,NaN,NaN,NaN,1.725867e+12,2.534023e+14,1869397.0,3.0
4,9591903,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,DDOT,1,2021-01-01 06:05:25,2021-01-26 21:40:13,2021-01-12 06:05:25,2021-01-01 06:05:25,...,NaN,{B7D25EBE-F302-E2F7-E053-542F520AD49B},NaN,NaN,NaN,NaN,1.725867e+12,2.534023e+14,1873581.0,25.0


In [4]:
servicecode_count = pd.DataFrame(features['servicecode'].value_counts()).reset_index().rename(columns={'count': 'servicecode_count'})

In [5]:
service_codes = features.drop_duplicates('servicecode')[['servicecode', 'servicecodedescription', 'servicetypecodedescription']]

In [6]:
service_codes.shape

(137, 3)

In [7]:
service_codes = service_codes.merge(servicecode_count, how='left', on='servicecode')

In [8]:
service_codes.shape

(137, 4)

In [9]:
service_codes.head()

,servicecode,servicecodedescription,servicetypecodedescription,servicecode_count
0,S0031,Bulk Collection,SWMA- Solid Waste Management Admistration,205502
1,BICYCLE,Abandoned Bicycle,SWMA- Solid Waste Management Admistration,2212
2,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,40069
3,S0441,Trash Collection - Missed,SWMA- Solid Waste Management Admistration,50492
4,S0261,Parking Enforcement,PEMA- Parking Enforcement Management Administr...,152390


### Add service codes to `request_categories.json`

In [10]:
with open('../streamlit_app/request_categories.json') as f:
    category_dict = json.load(f)

In [11]:
service_codes.index = service_codes["servicecodedescription"]
service_code_dict = service_codes.to_dict(orient="index")

In [12]:
descs_to_check = []
num_descs = 0
for category_str in category_dict:
    cat_dict = category_dict[category_str]
    cat_dict = dict(sorted(cat_dict.items()))
    for desc in cat_dict:
        if cat_dict[desc] is None:
            if desc in service_code_dict:
                cat_dict[desc] = service_code_dict[desc]['servicecode']
            else:
                descs_to_check.append(desc)
            num_descs += 1
    category_dict[category_str] = cat_dict

In [13]:
cat_dict

{'Alley Cleaning': 'S0011',
 'Bulk Collection': 'S0031',
 'Container Removal': 'CONTREMO',
 'DOEE - Reporting Electronics in Trash': 'REIT121918',
 'Dead Animal Collection': '11',
 'Graffiti Removal': 'GRAFF',
 'Illegal Dumping': 'S0181',
 'Illegal Poster': 'S0185',
 'Lost/Stolen Compost Bin, Broken Compost Bin or Opt-Out of Curbside Composting Pilot Program': 'COMPOST',
 'Missed Curbside Compost Collection': 'COMPOSTM',
 'Pet Waste Complaint': 'PETWASTECOMP',
 'Public Space Litter Can Installation': 'LCANINS',
 'Public Space Litter Can Removal': 'LCanRem',
 'Public Space Litter Can Repair': 'LCanRep',
 'Public Space Litter Can-Collection': 'S0217',
 'Rat Replacement Containers': 'RATREPLACECONTRS',
 'Recycling - School Program': 'RECSCHPR',
 'Recycling Cart - Repair': 'RECCONRE',
 'Recycling Cart Delivery': 'S0316',
 'Recycling Collection - Missed': 'S0321',
 'Recycling- Information Request': 'S0322',
 'Rodent Inspection and Treatment': 'S0311',
 'Sanitation Enforcement': 'S0346',
 'S

In [14]:
with open('../streamlit_app/request_categories.json', 'w') as f:
    json.dump(category_dict, f, indent=4)